<a href="https://colab.research.google.com/github/Amnariaz640/Pytorch-/blob/main/Flask_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install flask-ngrok
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 874.0/874.0 kB 13.9 MB/s eta 0:00:00


In [ ]:
!pip install pyngrok

In [3]:
import os
from flask import Flask, request, send_from_directory, jsonify
from flask_ngrok import run_with_ngrok
from werkzeug.utils import secure_filename
from ultralytics import YOLO
from pyngrok import ngrok

# Create the Flask app
app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when the app runs

# Set the directory to save uploaded images
UPLOAD_FOLDER = '/content/drive/MyDrive/yoylov8/Image_Upload'
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

# Make sure the folder exists
os.makedirs(UPLOAD_FOLDER, exist_ok=True)

# Load your trained YOLOv8 model
model = YOLO('/content/drive/MyDrive/yolov8/Save_Result/Workers3/weights/best.pt')

# HTML upload form
@app.route('/')
def upload_form():
    return '''
    <h1>Upload an Image for YOLOv8 Prediction</h1>
    <form method="post" action="/predict" enctype="multipart/form-data">
      <input type="file" name="file" required>
      <input type="submit">
    </form>
    '''

# Handle the image upload and make prediction
@app.route('/predict', methods=['POST'])
def upload_image():
    if 'file' not in request.files:
        return 'No file uploaded', 400

    file = request.files['file']
    if file.filename == '':
        return 'No selected file', 400

    filename = secure_filename(file.filename)
    file_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
    file.save(file_path)

    # Run the model on the uploaded image
    results = model(file_path)

    # Save the result image with predictions
    annotated_image_path = os.path.join(app.config['UPLOAD_FOLDER'], f'annotated_{filename}')
    results[0].save(annotated_image_path)

    # Generate prediction resuls
    predictions = []
    for box in results[0].boxes:
        class_id = int(box.cls)  # Get the class ID
        confidence = box.conf.item() # Get the confidence score
        predictions.append((results[0].names[class_id], confidence))  # Map class ID to name

    # Format predictions for display
    predictions_str = ', '.join([f"{name} ({conf:.2f})" for name, conf in predictions])

    # Generate the public URL for the annotated image
    public_url = f"{request.host_url}uploads/annotated_{filename}"

    return f'''
    <h1>Prediction Result</h1>
    <p>Predictions: {predictions_str}</p>
    <img src="{public_url}" alt="Annotated Image">
    '''

# Serve the uploaded files
@app.route('/uploads/<filename>')
def uploaded_file(filename):
    return send_from_directory(app.config['UPLOAD_FOLDER'], filename)

# Authenticate with your ngrok authtoken
ngrok.set_auth_token("2mSexs9ABUBIZ2YOfAt6Jyd1urZ_2QaumHDiwqvEyxdem2D4i")

# Start the Flask app
if __name__ == "__main__":
    public_url = ngrok.connect(5000)  # ngrok will connect to the default Flask port
    print(f"Public URL: {public_url}")
    app.run()

Public URL: NgrokTunnel: "https://4fc5-35-221-211-10.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://4fc5-35-221-211-10.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [23/Sep/2024 09:32:43] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Sep/2024 09:32:44] "GET /favicon.ico HTTP/1.1" 404 -



image 1/1 /content/drive/MyDrive/yoylov8/Image_Upload/005309_jpg.rf.7857830095f72844b36a8bda9f820a12.jpg: 480x640 2 helmets, 665.3ms
Speed: 6.9ms preprocess, 665.3ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)


...

INFO:werkzeug:127.0.0.1 - - [23/Sep/2024 09:34:44] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Sep/2024 09:34:45] "GET /uploads/annotated_005298_jpg.rf.647d148af5d961d8bbc041f172247170.jpg HTTP/1.1" 200 -



image 1/1 /content/drive/MyDrive/yoylov8/Image_Upload/005325_jpg.rf.45a2fcd397555b21cb09401d1a044d25.jpg: 480x640 8 heads, 508.7ms
Speed: 2.8ms preprocess, 508.7ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)


INFO:werkzeug:127.0.0.1 - - [23/Sep/2024 09:35:05] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Sep/2024 09:35:06] "GET /uploads/annotated_005325_jpg.rf.45a2fcd397555b21cb09401d1a044d25.jpg HTTP/1.1" 200 -



image 1/1 /content/drive/MyDrive/yoylov8/Image_Upload/005304_jpg.rf.68023ed553bb1d711826d6e1f185ea39.jpg: 640x480 4 helmets, 447.3ms
Speed: 3.1ms preprocess, 447.3ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 480)


INFO:werkzeug:127.0.0.1 - - [23/Sep/2024 09:35:51] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [23/Sep/2024 09:35:52] "GET /uploads/annotated_005304_jpg.rf.68023ed553bb1d711826d6e1f185ea39.jpg HTTP/1.1" 200 -
